In [182]:
# Imports

import pandas as pd
import os
import csv

In [183]:
# Funções auxiliares

def verificar_e_imprimir_numeros(data_frame, nome_da_marca):
    numeric_values = pd.to_numeric(data_frame['ano'], errors='coerce')
    todos_numeros = numeric_values.notna().all()
    print(f'Todos os valores na coluna "ano" são números no dataset {nome_da_marca}? {todos_numeros}')

    if not todos_numeros:
        linhas_nao_numericas = data_frame[~numeric_values.notna()]
        print('\nValores não numéricos na coluna "ano":')
        for indice, valor in linhas_nao_numericas['ano'].items():
            print(f'Índice: {indice}, Valor: {valor}')

    return todos_numeros

def ler_csv_para_dataframe(caminho_arquivo_csv):
    with open(caminho_arquivo_csv, 'r', newline='', encoding='utf-8') as arquivo_csv:
        leitor_csv = csv.reader(arquivo_csv)
        linhas = list(leitor_csv)        
    df = pd.DataFrame(linhas[1:], columns=linhas[0])
    return df
    
def renomear_arquivos_na_pasta(pasta, palavra_antiga, palavra_nova):
    for nome_arquivo in os.listdir(pasta):
        caminho_arquivo_antigo = os.path.join(pasta, nome_arquivo)
        if palavra_antiga in nome_arquivo:
            novo_nome_arquivo = nome_arquivo.replace(palavra_antiga, palavra_nova)
            caminho_arquivo_novo = os.path.join(pasta, novo_nome_arquivo)
            os.rename(caminho_arquivo_antigo, caminho_arquivo_novo)
    print("Arquivos renomeados com sucesso!")

def obter_nomes_apos_substring(pasta):
    nomes_extraidos = []
    for nome_arquivo in os.listdir(pasta):
        if "Scraped_Car_Review_" in nome_arquivo:
            parte_nome = nome_arquivo.split("Scraped_Car_Review_")[1].replace(".csv", "")
            nomes_extraidos.append(parte_nome)
    return nomes_extraidos

In [184]:
# Funcoes de tratamento de dados

def tratarValorDoAno(dataFramePAno, marca):
    dataFramePAno['ano'] = dataFramePAno['Vehicle_Title'].str.extract('(\\d{4})')
    numeric_values = pd.to_numeric(dataFramePAno[''], errors='coerce')
    dataFramePAno = dataFramePAno[pd.notna(numeric_values)]
    allNum = verificar_e_imprimir_numeros(dataFramePAno, marca)
    return dataFramePAno

In [185]:
# Tratamento dos arquivos nas pastas

pasta = './Data/'
palavra_antiga = "Reviews"
palavra_nova = "Review"
renomear_arquivos_na_pasta(pasta, palavra_antiga, palavra_nova)

palavra_antiga = "Scrapped"
palavra_nova = "Scraped"
renomear_arquivos_na_pasta(pasta, palavra_antiga, palavra_nova)



Arquivos renomeados com sucesso!
Arquivos renomeados com sucesso!


In [186]:
# Upload dos datasets

marcas_de_carro = obter_nomes_apos_substring("./Data/")
todos_df = []

for marca in marcas_de_carro:
    file_path = f'./Data/Scraped_Car_Review_{marca}.csv'
    df = ler_csv_para_dataframe(file_path)
    todos_df.append(df)

df_final = pd.concat(todos_df, ignore_index=True)
df_final = tratarValorDoAno(df_final, marca)


caminho_arquivo_final = './arquivo_final.xlsx'
df_final.to_excel(caminho_arquivo_final, index=False)
num_linhas = df_final.shape[0]
print(f'O arquivo XLSX tem {num_linhas} linhas.')
    
    


Todos os valores na coluna "ano" são números no dataset AstonMartin? False

Valores não numéricos na coluna "ano":
Índice: 58862, Valor: nan
Índice: 80018, Valor: nan
Índice: 144173, Valor: nan
Índice: 179279, Valor: nan
Índice: 193865, Valor: nan
Índice: 202453, Valor: nan
Índice: 233018, Valor: nan
Índice: 261609, Valor: nan
O arquivo XLSX tem 226939 linhas.
